In [ ]:
#This code is used to align the 2 images features 

from __future__ import print_function
import cv2
import numpy as np
 
MAX_FEATURES = 450
GOOD_MATCH_PERCENT = 0.15
 
def alignImages(im1, im2):
 
  # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
   
  # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
   
  # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
   
  # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]
 
  # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite("matches.jpg", imMatches)
    #cv2.imshow("matches.jpg", imMatches)
  # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
   
  # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))
   
    return im1Reg, h
 
 
if __name__ == '__main__':
   
  # Read reference image
    refFilename = "C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Results/data(Total n_frames)/frame19.jpg"
    #print("Reading reference image : ", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
 
  # Read image to be aligned
    imFilename = "C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Results/data(Total n_frames)/frame20.jpg"
    #print("Reading image to align : ", imFilename);  
    im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
   
    print("Aligning images ...")
  # Registered image will be resotred in imReg. 
  # The estimated homography will be stored in h. 
    imReg, h = alignImages(im, imReference)
   
  # Write aligned image to disk. 
    outFilename = "C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Results/data_consecutive_out/Frame19.jpg"
    #print("Saving aligned image : ", outFilename); 
    cv2.imwrite(outFilename, imReg)
    #cv2.imshow("1",imReg)
    cv2.waitKey(0)
 
  # Print estimated homography
#print("Estimated homography : \n",  h)

In [ ]:
#This code is used to align the images in a video :(getting error)

import numpy as np
import cv2

cap = cv2.VideoCapture("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Original_video.avi")
print("capture= ",cap)
      
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("n_frames= ",n_frames)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print("width= ", width)
print("height= ", height)


fps = cap.get(cv2.CAP_PROP_FPS)
print("fps= ",fps)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
print("Fourcc= ",fourcc)

MAX_FEATURES = 200
GOOD_MATCH_PERCENT = 0.15


for i in range(0,1936):
    ret,prev_frame=cap.read()
    prev_gray=cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(prev_gray, None)
    for j in range(1,1937):
        succ, curr_frame = cap.read()
        curr_gray=cv2.cvtColor(curr_frame,cv2.COLOR_BGR2GRAY)
        keypoints2, descriptors2 = orb.detectAndCompute(curr_gray, None)
        matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
        matches = matcher.match(descriptors1, descriptors2, None)
        matches.sort(key=lambda x: x.distance, reverse=False)
        numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
        matches = matches[:numGoodMatches]
        frame_Matches = cv2.drawMatches(prev_frame, keypoints1, curr_frame, keypoints2, matches, None)
        #cv2.imwrite("matches.jpg", frame_Matches)
        cv2.imshow("matches.jpg",  frame_Matches)
        points1 = np.zeros((len(matches), 2), dtype=np.float32)
        points2 = np.zeros((len(matches), 2), dtype=np.float32)
        for z, match in enumerate(matches):
            points1[z, :] = keypoints1[match.queryIdx].pt
            #print(points1[z, :])
            points2[z, :] = keypoints2[match.trainIdx].pt
            #print(points2[z, :])
        h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
        print("h= ",h)
        height, width, channels = curr_frame.shape
        print(curr_frame.shape)
        frame_Reg = cv2.warpPerspective(prev_frame, h, (width, height))
        cv2.imshow("frame_out",frame_Reg)
if cv2.waitKey(1) & 0xFF==ord("q"):
            break
   
 
cap.release()
#out.release()
cv2.destroyAllWindows()      
        
    


In [ ]:
#Basic code to implement hough transform on a image
import cv2
import numpy as np


cap=cv2.VideoCapture("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Results/cropped_video.avi")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#print("Fourcc= ",fourcc)
fps=4.0
#n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#print("n_frames= ",n_frames)

#out = cv2.VideoWriter('Hough_line_video.avi',fourcc, fps, (1280, 720),True)
#print("Ouput= ",out)
while(True):
    ret,frame=cap.read()
    #frame = cv2.imread("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Results/data_out/Frame1.jpg")
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray,50,150,apertureSize = 3)

    lines = cv2.HoughLines(edges,1,np.pi/180,20)
    for rho,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
    cv2.line(frame,(390,240),(390,500),(0,255,0),2)
    #cv2.line(frame,(640,460),(640,720),(0,0,255),2)
    cv2.imshow("line",frame)
    #out.write(frame)
    #cv2.line(img,(480,460),(480,720),(0,0,255),2)
    #cv2.line(img,(780,460),(780,720),(0,0,255),2)
    if cv2.waitKey(1) & 0xFF==ord("q"):
            break
cap.release()
out.release()
cv2.destroyAllWindows()
    
    
#cv2.imwrite("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Results/Hough_lines_data_out/Frame2.jpg",img)

In [ ]:
# This code is used to reconstruct the video using the frames
import cv2
import numpy as np
import glob
 
img_array = []
for filename in glob.glob("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Results/data_consecutive_out/*.jpg"):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Results/data_consecutive_out/video.avi",cv2.VideoWriter_fourcc(*'XVID'), 2, (1280,720))
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
#CODE to crop a video
import numpy as np
import cv2

cap=cv2.VideoCapture("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Results/data_out/video.avi")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
print("Fourcc= ",fourcc)
fps=4.0

out = cv2.VideoWriter('Align&crop_video.avi',fourcc, fps, (780, 500),True)
print("Ouput= ",out)
while(True):
    ret,frame=cap.read()
    cropped=frame[100:600,220:1000]
    print(cropped.size)
    print(cropped.shape)
    cv2.imshow("cropped",cropped)
    out.write(cropped)
    if cv2.waitKey(1) & 0xFF==ord("q"):
            break
cap.release()
out.release()
cv2.destroyAllWindows()

In [1]:
#Code of applying Hough Transform on a video

import cv2
import numpy as np
cap=cv2.VideoCapture("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Original_video.avi")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
print("Fourcc= ",fourcc)
fps=30.0
out = cv2.VideoWriter('Tracking_Railway_lines_Original_video.avi',fourcc, fps, (1280, 720),True)
while(True):
    ret,frame=cap.read()
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame1=cv2.GaussianBlur(hsv,(21,21),0,0)
    cv2.imshow("hsv",hsv)
    low_pink=np.array([50, 104, 144])
    up_pink=np.array([180,  255, 255 ])
    
    Mask=cv2.inRange(frame,low_pink,up_pink)
    #res=cv2.bitwise_and(frame,frame,Mask)
    edges=cv2.Canny(Mask,75,150)
    lines=cv2.HoughLinesP(edges,1,np.pi/180,20)
    
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2=line[0]
            cv2.line(frame,(x1,y1),(x2,y2),(0,255,0),2)
    print(lines)
    cv2.imshow("mask",Mask)
    #cv2.imshow("res",res)
    cv2.imshow("frame",frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF==ord("q"):
            break
cap.release()
out.release()
cv2.destroyAllWindows()
    

Fourcc=  1145656920
[[[ 531  576  531  575]]

 [[ 643  231  643  230]]

 [[ 744  473  744  473]]

 ...

 [[  84  236   84  236]]

 [[1143  331 1143  331]]

 [[ 919   21  919   21]]]
[[[ 752  515  752  517]]

 [[ 759  504  759  506]]

 [[ 722  358  722  359]]

 ...

 [[1228  238 1229  238]]

 [[  81  432   81  432]]

 [[ 105  470  105  470]]]
[[[ 716  285  716  285]]

 [[ 653  485  657  485]]

 [[  87  327   88  327]]

 ...

 [[1179  531 1181  530]]

 [[1126  619 1128  619]]

 [[ 937  369  938  369]]]
[[[ 382  307  385  305]]

 [[ 264  378  265  377]]

 [[ 247  388  247  388]]

 ...

 [[1087  603 1087  603]]

 [[ 402  635  402  635]]

 [[ 638  583  638  583]]]
[[[ 380  289  380  289]]

 [[ 363  367  363  367]]

 [[  10  663   13  660]]

 ...

 [[1134  613 1134  613]]

 [[1142  549 1142  549]]

 [[ 691  325  691  325]]]
[[[  87  589   89  587]]

 [[1124  556 1124  556]]

 [[ 577  627  583  627]]

 ...

 [[ 424  703  425  703]]

 [[1011  208 1011  208]]

 [[ 872  101  873  101]]]
[[[728  

[[[ 745  103  763  103]]

 [[  93   24  119   24]]

 [[ 854  100  854  100]]

 ...

 [[ 591  572  591  572]]

 [[ 449  120  449  120]]

 [[1068   79 1068   77]]]
[[[ 752  298  752  300]]

 [[ 797  491  797  492]]

 [[ 795  482  795  484]]

 ...

 [[1187  508 1187  508]]

 [[ 340  106  340  106]]

 [[ 871   69  871   69]]]
[[[ 356   89  357   89]]

 [[ 601   88  611   88]]

 [[ 913   78  921   78]]

 ...

 [[1191  688 1192  688]]

 [[ 770  448  770  447]]

 [[ 144  436  144  436]]]
[[[ 754   16  767   16]]

 [[ 739  100  741  100]]

 [[ 593  115  598  115]]

 ...

 [[1134  123 1134  123]]

 [[ 740  484  740  485]]

 [[ 300  121  301  121]]]
[[[ 111   39  137   39]]

 [[ 750  638  750  640]]

 [[   3  648    4  647]]

 ...

 [[ 820  115  820  115]]

 [[1269  195 1269  193]]

 [[1118  705 1118  705]]]
[[[ 756  592  756  594]]

 [[ 155  129  155  129]]

 [[ 779  692  779  694]]

 ...

 [[1061  711 1061  711]]

 [[1131  472 1132  473]]

 [[ 407  121  408  121]]]
[[[ 882   76  899   76]]

 [

[[[ 727  134  749  134]]

 [[ 273   51  273   51]]

 [[ 674   38  687   38]]

 ...

 [[ 618  357  618  357]]

 [[ 852  189  852  189]]

 [[1255  637 1256  637]]]
[[[329  32 332  32]]

 [[218  32 247  32]]

 [[291 157 291 157]]

 ...

 [[565 171 565 171]]

 [[551 119 551 119]]

 [[793 190 793 190]]]
[[[ 763   32  765   32]]

 [[ 705  479  705  481]]

 [[   8   45   20   45]]

 ...

 [[1177  627 1177  627]]

 [[  15  526   15  526]]

 [[ 721  531  721  532]]]
[[[ 618   24  618   24]]

 [[ 555   27  569   27]]

 [[ 701  472  701  472]]

 ...

 [[1266  589 1266  589]]

 [[1221  106 1221  105]]

 [[ 984  673  985  673]]]
[[[ 221   54  221   54]]

 [[ 556  133  571  133]]

 [[ 540   43  552   43]]

 ...

 [[ 558   52  558   52]]

 [[  59   43   59   43]]

 [[1191  582 1191  581]]]
[[[ 862  120  882  120]]

 [[  79   39   93   39]]

 [[ 567  516  567  515]]

 ...

 [[  15  450   15  450]]

 [[ 613  372  613  371]]

 [[1211  621 1212  622]]]
[[[ 397  132  414  132]]

 [[ 397   21  413   21]]



[[[ 735  548  735  550]]

 [[  76   28   92   28]]

 [[1270   92 1270   92]]

 ...

 [[ 527  351  527  351]]

 [[ 383  126  384  126]]

 [[ 895  150  895  149]]]
[[[ 845  106  845  106]]

 [[ 552  116  556  116]]

 [[ 456  124  470  124]]

 ...

 [[ 785   12  785   12]]

 [[1058  717 1058  716]]

 [[ 723  161  723  161]]]
[[[ 709  115  733  115]]

 [[ 518  129  520  129]]

 [[ 511  119  529  119]]

 ...

 [[1115  659 1115  659]]

 [[ 613  404  613  404]]

 [[1168  129 1168  129]]]
[[[863 107 886 107]]

 [[427  16 441  16]]

 [[502  18 513  18]]

 ...

 [[422 171 424 171]]

 [[404 342 404 342]]

 [[527 681 527 679]]]
[[[ 719  467  719  469]]

 [[ 568  117  584  117]]

 [[ 757  108  773  108]]

 ...

 [[1023  150 1023  150]]

 [[1109  106 1109  107]]

 [[ 906   86  906   86]]]
[[[719 116 740 115]]

 [[802 133 809 133]]

 [[714 434 714 436]]

 ...

 [[972  66 972  66]]

 [[123 682 123 682]]

 [[697  76 699  78]]]
[[[757  99 773  99]]

 [[719 102 721 102]]

 [[514 110 525 110]]

 ...

 [[5

[[[ 979  156  979  156]]

 [[1270  125 1271  125]]

 [[ 438  156  441  156]]

 ...

 [[ 724  121  724  121]]

 [[ 901  233  901  233]]

 [[1012  154 1012  154]]]
[[[758 154 760 154]]

 [[570  15 598  15]]

 [[907 140 907 140]]

 ...

 [[556 184 558 184]]

 [[996 169 996 169]]

 [[730 508 730 508]]]
[[[ 817  139  832  139]]

 [[ 580  157  592  157]]

 [[ 621  158  629  158]]

 ...

 [[1059  498 1059  498]]

 [[ 122   21  123   21]]

 [[1187  677 1187  677]]]
[[[377 150 380 150]]

 [[432 148 435 148]]

 [[780 135 780 135]]

 ...

 [[137 195 137 195]]

 [[711 395 711 395]]

 [[253 126 253 129]]]
[[[ 264   33  287   33]]

 [[ 252   32  256   32]]

 [[   5   37   14   37]]

 ...

 [[ 529   33  529   33]]

 [[1126  179 1127  179]]

 [[ 872  368  872  368]]]
[[[665   7 678   7]]

 [[749 148 758 148]]

 [[266 166 267 166]]

 ...

 [[546 176 546 176]]

 [[155 454 155 454]]

 [[163 585 164 584]]]
[[[ 482    9  499    9]]

 [[ 560    8  597    8]]

 [[  63   41   74   41]]

 ...

 [[  30   21   3

...

[[[962  68 962  96]]

 [[442 101 442 101]]

 [[866 176 866 176]]

 ...

 [[597 319 597 319]]

 [[726 204 726 204]]

 [[966 290 966 290]]]
[[[ 102   42  102   42]]

 [[ 976  177  976  207]]

 [[ 978  368  978  369]]

 ...

 [[1172  625 1172  625]]

 [[ 139    7  139    7]]

 [[1153  657 1154  658]]]
[[[ 749  165  749  165]]

 [[ 120   43  120   43]]

 [[ 148  548  148  548]]

 ...

 [[ 951  121  951  121]]

 [[1088  680 1088  680]]

 [[ 561  322  561  322]]]
[[[ 974   36  974   59]]

 [[1170  439 1171  439]]

 [[ 331   80  331   80]]

 ...

 [[ 190   88  190   88]]

 [[ 482  673  482  673]]

 [[1007  604 1007  604]]]
[[[ 828  166  828  195]]

 [[  70   20   72   21]]

 [[1103  363 1107  364]]

 ...

 [[ 965  473  965  473]]

 [[ 162   57  162   57]]

 [[  90  583   92  582]]]
[[[ 186  530  187  529]]

 [[ 560  160  560  160]]

 [[1210  379 1210  379]]

 ...

 [[ 185   30  185   30]]

 [[1030  188 1030  188]]

 [[ 956  430  956  430]]]
[[[ 987  217  987  218]]

 [[ 977  319  979  319]]



[[[ 571  668  576  668]]

 [[1077  422 1077  422]]

 [[1011  345 1011  345]]

 ...

 [[ 713   90  713   90]]

 [[1211  578 1211  578]]

 [[ 610  599  610  599]]]
[[[ 397  528  399  528]]

 [[ 296  374  298  373]]

 [[ 662  506  662  506]]

 ...

 [[1183  489 1183  489]]

 [[1202  389 1203  390]]

 [[ 540  658  542  658]]]
[[[ 516  564  516  564]]

 [[ 601  561  621  561]]

 [[ 830  373  836  372]]

 ...

 [[  92  500   93  499]]

 [[1076  434 1076  434]]

 [[ 357  493  357  493]]]
[[[ 598  486  599  485]]

 [[ 985  194  985  194]]

 [[1009  335 1009  335]]

 ...

 [[1006  257 1006  257]]

 [[ 527  520  528  517]]

 [[ 358  361  359  361]]]
[[[ 680  506  694  506]]

 [[ 272  592  272  592]]

 [[ 184  519  199  519]]

 ...

 [[1187  318 1187  318]]

 [[1063   10 1063   10]]

 [[ 379  331  381  330]]]
[[[ 755  504  755  504]]

 [[1260  439 1260  439]]

 [[1229  521 1232  521]]

 ...

 [[ 300  564  300  564]]

 [[ 883  252  883  251]]

 [[1266  331 1266  330]]]
[[[ 602  675  621  675]]

 [

[[[ 326  451  326  451]]

 [[ 349  437  350  436]]

 [[ 571  522  577  522]]

 ...

 [[1101  453 1101  453]]

 [[ 762  280  763  279]]

 [[1087  295 1087  295]]]
[[[ 153  689  154  688]]

 [[ 853  145  853  145]]

 [[ 289  571  294  566]]

 ...

 [[ 833  677  833  677]]

 [[ 187  181  187  181]]

 [[1112  455 1113  454]]]
[[[ 640  654  648  654]]

 [[1046  256 1047  256]]

 [[ 688  519  691  519]]

 ...

 [[ 513  102  513  102]]

 [[1254  132 1254  132]]

 [[ 128  612  128  612]]]
[[[1206   66 1207   51]]

 [[ 397   26  397   26]]

 [[ 153  536  153  536]]

 ...

 [[  18  354   18  354]]

 [[1269  250 1269  250]]

 [[ 185  509  185  509]]]
[[[ 861  151  862  150]]

 [[ 241  541  250  541]]

 [[ 570  683  579  683]]

 ...

 [[ 916  154  916  154]]

 [[1265   98 1265   98]]

 [[1143  329 1143  329]]]
[[[ 670  662  680  662]]

 [[1041  265 1041  265]]

 [[  94  261   95  261]]

 ...

 [[1019  325 1019  325]]

 [[ 984  380  985  379]]

 [[ 860  156  860  156]]]
[[[ 235  498  238  496]]

 [

[[[ 294  480  294  480]]

 [[ 533  685  533  683]]

 [[ 327  460  327  460]]

 ...

 [[1227   61 1227   60]]

 [[ 640  628  642  628]]

 [[ 593  603  594  602]]]
[[[ 981  163  984  163]]

 [[ 719  272  719  272]]

 [[1115  173 1118  173]]

 ...

 [[   2  640    2  640]]

 [[1020  361 1022  361]]

 [[1135  502 1135  502]]]
[[[1207   66 1207   62]]

 [[ 344  441  344  441]]

 [[ 437   62  437   62]]

 ...

 [[ 576   98  576   98]]

 [[1072  713 1074  713]]

 [[ 367   93  368   93]]]
[[[ 196  603  196  603]]

 [[ 153  551  155  550]]

 [[ 760  141  760  141]]

 ...

 [[1122  226 1122  226]]

 [[ 359   71  359   71]]

 [[1176  200 1176  200]]]
[[[ 162  653  176  653]]

 [[1015  348 1016  348]]

 [[ 303   31  305   31]]

 ...

 [[ 496   70  496   70]]

 [[ 866  158  866  158]]

 [[ 821  137  821  137]]]
[[[ 341  439  342  438]]

 [[ 641  641  641  641]]

 [[1155   89 1155   81]]

 ...

 [[ 373  625  373  625]]

 [[ 906  323  906  323]]

 [[ 856  288  858  288]]]


error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\color.cpp:11214: error: (-215) (scn == 3 || scn == 4) && (depth == 0 || depth == 5) in function cv::cvtColor
